In [1]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[0.1123],
        [0.1110]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
net[2].state_dict(),net.state_dict()

(OrderedDict([('weight',
               tensor([[ 0.1952,  0.1933,  0.3364,  0.2534,  0.1545,  0.2872, -0.1041, -0.0663]])),
              ('bias', tensor([-0.2922]))]),
 OrderedDict([('0.weight',
               tensor([[ 0.4259, -0.2544,  0.3114, -0.0191],
                       [ 0.2780,  0.3777,  0.2822, -0.2920],
                       [-0.0065,  0.2706,  0.1803,  0.2921],
                       [-0.0469, -0.3680, -0.2666, -0.2596],
                       [ 0.0953,  0.3712, -0.2075, -0.0467],
                       [ 0.0141,  0.2735,  0.2935,  0.4018],
                       [ 0.3295,  0.3687,  0.3892, -0.1069],
                       [-0.3475, -0.0948, -0.1918,  0.1337]])),
              ('0.bias',
               tensor([-0.4022,  0.2000,  0.2616, -0.0250,  0.2335,  0.1191,  0.1461, -0.0615])),
              ('2.weight',
               tensor([[ 0.1952,  0.1933,  0.3364,  0.2534,  0.1545,  0.2872, -0.1041, -0.0663]])),
              ('2.bias', tensor([-0.2922]))]))

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
net[2].weight.grad==None

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2922], requires_grad=True)
tensor([-0.2922])


True

In [4]:
print(net[0].named_parameters())
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])
net.state_dict()['2.bias'].data

<generator object Module.named_parameters at 0x0000022FCE85E040>
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([-0.2922])

In [5]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                         nn.Linear(8,4),nn.ReLU())
def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [6]:
rgnet(X),rgnet[0][1][0].bias.data

(tensor([[0.1071],
         [0.1071]], grad_fn=<AddmmBackward0>),
 tensor([-0.0297,  0.0988, -0.0752,  0.4339,  0.2289, -0.4404, -0.1611, -0.1208]))

参数初始化

In [7]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0098,  0.0114, -0.0095, -0.0040]), tensor(0.))

In [8]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [9]:
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
net[0].weight.data[0],net[2].weight.data

(tensor([-0.1456, -0.5308, -0.4543, -0.0233]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [10]:
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data*=m.weight.data.abs()>=5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.9181, -0.0000, -7.4324,  0.0000],
        [-0.0000, -0.0000, -7.6489, -0.0000]], grad_fn=<SliceBackward0>)

In [11]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -6.4324,  1.0000])

共享参数

In [12]:
shared=nn.Linear(8,8)   # 我们需要给共享层一个名称,以便可以引用它的参数
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  shared,nn.ReLU(),
                  shared,nn.ReLU(),
                  nn.Linear(8,1))
net(X)
print(net[2].weight.data[0]==net[4].weight.data[0]) # 检查参数是否相同
net[2].weight.data[0,0]=100
print(net[2].weight.data[0]==net[4].weight.data[0]) # 确保它们实际上是同一个对象,而不只是有相同的值

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


Exercises

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear=nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))

chimera=NestMLP()
{k:v.shape for k,v in chimera.state_dict().items()}

{'net.0.weight': torch.Size([64, 20]),
 'net.0.bias': torch.Size([64]),
 'net.2.weight': torch.Size([32, 64]),
 'net.2.bias': torch.Size([32]),
 'linear.weight': torch.Size([16, 32]),
 'linear.bias': torch.Size([16])}

In [14]:
chimera.state_dict()['net.2.weight'][0][0]==chimera.net[2].weight[0][0],\
chimera.state_dict()['linear.weight'][0][0]==chimera.linear.weight[0][0]

(tensor(True), tensor(True))

训练包含共享参数层的多层感知机（以多项式回归为例）

In [43]:
import math
import numpy as np
from d2l import torch as d2l

n_train,n_test=200,100
true_w=np.zeros(4)
true_w[0:4]=np.array([5,1.2,-3.4,5.6])

features=np.random.normal(size=(n_train+n_test,1))
np.random.shuffle(features)
poly_features=np.power(features,np.arange(4).reshape(1,-1))
for i in range(4):
    poly_features[:,i]/=math.gamma(i+1)
labels=np.dot(poly_features,true_w)
labels+=np.random.normal(scale=0.1,size=labels.shape)
true_w,features,poly_features,labels=[torch.tensor(x,dtype=torch.float32) for x in [true_w,features,poly_features,labels]]

def train(train_features,test_features,train_labels,test_labels,num_epochs=400):
    loss=nn.MSELoss()
    shared=nn.Linear(8,8,bias=False)
    net=nn.Sequential(nn.Linear(4,8,bias=False),nn.ReLU(),
                      shared,nn.ReLU(),shared,nn.ReLU(),
                      nn.Linear(8,1,bias=False))
    batch_size=min(10,train_labels.shape[0])
    train_iter=d2l.load_array((train_features,train_labels.reshape(-1,1)),batch_size)
    test_iter=d2l.load_array((test_features,test_labels.reshape(-1,1)),batch_size,is_train=False)
    trainer=torch.optim.SGD(net.parameters(),lr=0.01)
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net,train_iter,loss,trainer)
    for i in [0,2,4,6]: # 结果可以看到共享层参数的值和梯度是相同的
        print('layer',i//2,' weight.data[0]:',net[i].weight.data[0],' weight.bias[0]:',net[i].weight.grad[0])

train(poly_features[:n_train,:4],poly_features[n_train:,:4],labels[:n_train],labels[n_train:])

layer 0  weight.data[0]: tensor([-0.3648,  0.2111, -0.3065, -0.0989])  weight.bias[0]: tensor([0., 0., 0., 0.])
layer 1  weight.data[0]: tensor([-0.1947,  0.0518,  0.0532, -0.3742, -0.0641, -0.0398,  0.7691, -0.2661])  weight.bias[0]: tensor([ 0.0072, -0.0011,  0.0171,  0.0041,  0.0020,  0.0342,  0.0062,  0.0091])
layer 2  weight.data[0]: tensor([-0.1947,  0.0518,  0.0532, -0.3742, -0.0641, -0.0398,  0.7691, -0.2661])  weight.bias[0]: tensor([ 0.0072, -0.0011,  0.0171,  0.0041,  0.0020,  0.0342,  0.0062,  0.0091])
layer 3  weight.data[0]: tensor([-0.7376,  0.2640,  0.1476, -0.5621,  0.2634, -0.9461,  1.1562, -0.2565])  weight.bias[0]: tensor([ 0.0088,  0.0000, -0.0078, -0.0023,  0.0000, -0.0079, -0.1378,  0.0000])
